Inicialmente, pegamos os dados brutos e vamos fazer uma filtragem para selecionar apenas as informações dos 18 principais estados que analisaremos.

In [ ]:
import pandas as pd

lista_estacao = []
lista_sigla = []
with open('codestations.txt') as f:
    for line in f:
        lista_estacao.append(line[:11])
        lista_sigla.append(line[38:40])

df = pd.DataFrame({'estacao': lista_estacao, 'sigla': lista_sigla})
df_eua = df[df['estacao'].str.startswith('US')]


#data must be downloaded from  https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/ and saved in the folder ../NOAA data/NOAA - Dados Brutos/

for i in range(2023, 2024):
    chunk_size = 50000

    chunks = pd.read_csv(f'../../data/raw_data_NOAA/new_path/{i}.csv', header=None, chunksize=chunk_size)

    df_list = []
    for chunk in chunks:
        df_list.append(chunk)

    df = pd.concat(df_list)
    df_m = df[df[0].str.startswith('US')]

    df = pd.merge(df_m, df_eua, left_on=0, right_on='estacao', how='right')
    df.to_csv(f'../../NOAA data/NOAA - Dados Tratados/CSV - Por ano/{i}_TOP18.csv', index=False, header=False)

Aqui, pegamos os csv's gerados por ano já filtrados com os 18 estados de interesse e separamos cada estado

In [1]:
#dfs = {state: pd.DataFrame() for state in ['IA', 'IL', 'MN', 'NE', 'IN', 'SD', 'OH', 'WI', 'KS', 'MO', 'ND', 'MI', 'KY', 'TX', 'AR', 'CO', 'PA', 'TN']}

import pandas as pd
#ignore warnings
import warnings

warnings.filterwarnings("ignore")
dfs = {state: pd.DataFrame() for state in ['CO', 'PA', 'TN']} 
for i in range (1973, 2024): 
    df = pd.read_csv(f'../../data/raw_data_NOAA/by_year/{i}_TOP18.csv', header=None)
    
    for state in dfs.keys():
        df_state = df[df[9] == state]

        dfs[state] = pd.concat([dfs[state], df_state], ignore_index=True)

for state, df_state in dfs.items():
    df_state.to_csv(f'../../data/raw_data_NOAA/by_state/{state}.csv', index=False, header=False)


In [8]:
import pandas as pd
# ['IA', 'IL', 'MN', 'NE', 'IN', 'SD', 'OH', 'WI', 'KS', 'MO', 'ND', 'MI', 'KY', 'TX', 'AR', 'CO', 'PA', 'TN']
#['MN', 'IN', 'IL', 'IA'] ['AR', 'CO', 'KS', 'KY', 'MN']
siglas_estados = ['IA', 'IL', 'MN', 'NE', 'IN', 'SD', 'OH', 'WI', 'KS', 'MO', 'ND', 'MI', 'KY', 'TX', 'AR', 'CO', 'PA', 'TN']

for sigla in siglas_estados:
    df = pd.read_csv(f'../../data/raw_data_NOAA/by_state/{sigla}.csv', usecols=[0,1,2,3])
    df.columns = ['id', 'data', 'variavel', 'valor']

    df = df[df['variavel'].isin(['TMAX', 'TMIN', 'PRCP'])]
    df['data'] = pd.to_datetime(df['data'], format='%Y%m%d')

    df["mes"] = df["data"].dt.month
    df["ano"] = df["data"].dt.year

    df["quinzena"] = df["data"].apply(lambda x: 1 if x.day <= 15 else 2)
    # Pivot the table to have TMAX, TMIN, and PRCP as columns first
    df_pivot = df.pivot_table(index=['id', 'data', 'ano', 'mes', 'quinzena'], columns='variavel', values='valor', aggfunc='sum').reset_index()
 
    # Now, calculate the difference for each row
    df_pivot['diff'] = df_pivot['TMAX'] - df_pivot['TMIN']

    df_pivot['TMean'] = (df_pivot['TMAX'] + df_pivot['TMIN']) / 2 
    # Drop unnecessary columns for further processing
    df_pivot = df_pivot.drop(columns=['id'])
    
    # Now, group by year and month to calculate the mean
    df_media = df_pivot.groupby(["data"]).mean().reset_index()

    df_media = df_media[df_media['mes'].isin([5,6,7,8])]

    df_media = df_media.drop(columns=['quinzena'])

    df_monthly = df_media.groupby(['ano', 'mes']).agg({
        'PRCP': 'mean',
        'TMAX': 'mean',
        'TMIN': 'mean',
        'diff': 'mean',
        'TMean': 'mean',
    }).reset_index()

    df_monthly.to_csv(f'../../data/raw_data_NOAA/by_month_and_state/{sigla}.csv', index=False)



""" 
filtragem final para criar os dataframes de cada estado. Temos:
PRCP: precipitação
TMAX: temperatura máxima
TMIN: temperatura mínima
diff: diferença entre TMAX e TMIN
TMean: média entre TMAX e TMIN


Se quiser alterar quais estados terão dados gerados, troque a lista no começo do código. Lembre-se que é necessário ter realizado o tratamento do outro notebook antes de rodar este, 
caso não tenham sido gerados os arquivos CSVs de cada estado separadamente
"""

' \nfiltragem final para criar os dataframes de cada estado. Temos:\nPRCP: precipitação\nTMAX: temperatura máxima\nTMIN: temperatura mínima\ndiff: diferença entre TMAX e TMIN\nTMean: média entre TMAX e TMIN\n\n\nSe quiser alterar quais estados terão dados gerados, troque a lista no começo do código. Lembre-se que é necessário ter realizado o tratamento do outro notebook antes de rodar este, \ncaso não tenham sido gerados os arquivos CSVs de cada estado separadamente\n'